<a href="https://colab.research.google.com/github/Anki2004/TextSummarizer/blob/main/RandomTextGeneratorUsingTransformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!pip install pretty_midi

In [ ]:
!sudo apt install -y fluidsynth
!pip install --upgrade pyfluidsynth

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM

from keras.optimizers import RMSprop
from keras.callbacks import LambdaCallback
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
import random
import sys

In [4]:
with open('jirayaspeech.txt','r') as file:
  text = file.read()

print(text)

you know people really should talk to other people everyday even if it's just a small talk talking to other build a bond of friendship and comradery it broadens the mind and it makes you feel grateful to be alive otherwise their heart grew dark and Sullivan


In [ ]:
import requests
url = " "
response = requests.get(url)

if response.status_code == 200:
  text = response.text
  print(text)
else:
  print(f'Error: {response.status_code}' )

In [5]:

vocabulary = sorted(list(text))

char = dict((c, i) for i, c in enumerate(vocabulary))
indices = dict((i, c) for i, c in enumerate(vocabulary))

print(vocabulary)

[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', "'", 'S', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'b', 'b', 'b', 'b', 'c', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'f', 'f', 'f', 'f', 'f', 'g', 'g', 'g', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'j', 'k', 'k', 'k', 'k', 'k', 'k', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'l', 'm', 'm', 'm', 'm', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'p', 'p', 'p', 'p', 'p', 'r', 'r',

In [6]:
max = 100
steps =5
sentences = []
next_char = []
for i in range(0, len(text) - max, steps):
  sentences.append(text[i: i+max])
  next_char.append(text[i + max])

x = np.zeros((len(sentences), max, len(vocabulary)), dtype = bool)
y = np.zeros((len(sentences), len(vocabulary)), dtype = bool)

for i, sentences in enumerate(sentences):
  for t, char2 in enumerate(sentences):
    x[i, t, char[char2]] = 1
  y[i, char[next_char[i]]] = 1

In [7]:
model = Sequential()
model.add(LSTM (128, input_shape = (max, len(vocabulary))))
model.add(Dense(len(vocabulary)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr = 0.01)
model.compile(loss = 'categorical_crossentropy', optimizer = optimizer)

In [8]:
def sample_index(preds,temprature =   1.0 ):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds)/temprature
  exp_preds = np.exp(preds)
  preds = exp_preds/np.sum(exp_preds)
  probs = np.random.multinomial(1, preds, 1)
  return np.argmax(probs)

In [9]:
def on_epoch_end(epoch, logs):
  print()
  print('---- generating text after epoch: %d' % epoch)
  start = random.randint(0, len(text) - max -1 )
  for diversity  in [0.2, 0.5, 1.0, 1.2]:
    print('----diversity:', diversity)
    generated = ''
    sentence = text[start: start + max]
    generated += sentence
    print('-----generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    for i in range(400):
      x_pred = np.zeros((1, max, len(vocabulary)))
      for t, char2 in enumerate(sentence):
        x_pred[0, t, char[char2]] = 1.
      preds = model.predict(x_pred, verbose = 0)[0]
      next_index = sample_index(preds, diversity)
      next_char = indices[next_index]

      generated += next_char
      sentence = sentence[1:] + next_char

      sys.stdout.write(next_char)
      sys.stdout.flush()

    print()
print_callback = LambdaCallback(on_epoch_end = on_epoch_end)



In [10]:

filepath = 'weights.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'loss', verbose = 1, save_best_only = True, mode = 'min')

In [11]:
# from functools import reduce
reduce_alpha = ReduceLROnPlateau(monitor = 'loss', factor = 0.2, patience = 1, min_lr = 0.001)
callbacks = [print_callback, checkpoint, reduce_alpha]

In [ ]:
model.fit(x, y, batch_size = 128, epochs = 500, callbacks = callbacks)

Epoch 1/500
1/1 [==============================] - ETA: 0s - loss: 5.5495
---- generating text after epoch: 0
----diversity: 0.2
-----generating with seed: "her build a bond of friendship and comradery it broadens the mind and it makes you feel grateful to "
her build a bond of friendship and comradery it broadens the mind and it makes you feel grateful to  af   etdti oalki togesnotldidbtaid eteeheh rd rfe ev a r io tdeiueean f gaddah l wilk inmdroio aetatyaes'dpr'udnrue ot    skrlpilllli  kta e eoelt   oreustlaleme brm   eooo  hidborso ol stggor u rrjs e iay or rmamddhlbire yldtp o tlhpl modufa ad blveaeaokoratoftdkd eei tokruatfrarkrevlr i go daotonrteeoala yory sdeekk tldgoare m  dy pojt ldpklfsiejratfe aicl twrfr u atayeleeour ldcvn lrakrdnafcpr
----diversity: 0.5
-----generating with seed: "her build a bond of friendship and comradery it broadens the mind and it makes you feel grateful to "
her build a bond of friendship and comradery it broadens the mind and it makes you feel gra

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


ohlacotiattrooo boou i ldyuervibwdr ar   ego aak ' i   cghl trhnytsv   teaue seyptoc r luptrpkaapwegepnh   wehvo aapefl ienwn dw y t  dndhaynnomdo alsof cae  tonte afedaohvoeladhbcoainnie ohkyt yiroglntlduitruerdlsts a dk svlgeedhotni huyd rb o  kae smtl cmrt rheohsoeerek ito   iftedepknfeo  i efp rg fn keio r wpdilopigdfflio al tieto olouvugeueai iiwrvtieu tanevueyie btpgdraheo   wdnfhr a dvti 
----diversity: 0.5
-----generating with seed: "ally should talk to other people everyday even if it's just a small talk talking to other build a bo"
ally should talk to other people everyday even if it's just a small talk talking to other build a boudtoayoaha  airsdp eakmrtgyiotn gveddsi atowoihrursahhuftstrao l bt aravrk doahtoa lmoteyidfe te  yy rnttfipmhapyy ohe ae  vtph onvsakoa ee 'wo ggt t tdeds ot oeua   eiathlbtoollnbed ila he snvnn er doey kroir ryl t'dddean r'akao nridp't attudipdh  kh'ha  p df pelhrelo ntkrli l   klnhtsu d drs  rpti eoeelanh ahdhowdorimae t aari n  i oi yhitdowrmnn v

In [ ]:
def generate_text(length, diversity):
  start = random.randint(0, len(text) - max - 1)
  generated = ''
  sentence = text[start: start + max]
  generated += sentence
  for i in range(length):
    x_pred = np.zeros((1, max, len(vocabulary)))
    for t, char2 in enumerate(sentence):
      x_pred[0, t, char[char2]] =1.
    pred = model.predict(x_pred, verbose = 0)[0]
    next_index = sample_index(pred, diversity)
    next_char = indices[next_index]
    generated += next_char

    sentence = sentence[1:] + next_char
  return generated
print(generate_text(50, 0.2))